In [10]:
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Layer
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from tensorflow import keras
import keras.backend as kb

In [3]:
dataset = pickle.load(open("dataset/IMDB/dataset_IMDB.pickle", "rb"))

In [4]:
Y = np.array(list(map(lambda x: 1 if x=="positive" else 0, dataset["sentiment"])))

In [5]:
x_train,x_test,y_train,y_test = train_test_split(dataset["processed_review"],Y, test_size=0.1, shuffle=True)

In [6]:
def creazione_modello_GloVe(filename):
    f = open(filename, encoding="utf8")
    embeding_index = {}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeding_index[word] = coefs
    f.close()
    return embeding_index

In [7]:
embedding = creazione_modello_GloVe("dataset/glove.6B.50d.txt")

In [8]:
tokenizer = Tokenizer(num_words=142092)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

In [9]:
vocab_len = len(word_index)+1
embedding_vector_len = embedding["banan"].shape[0]
embedding_matrix = np.zeros((vocab_len, embedding_vector_len))

for word, index in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
        embedding_matrix[index, :] = vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embedding_vector_len, input_length=200, weights=[embedding_matrix])

## Creazione classe ad-hoc per implementare il meccanismo di attention

In [15]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=kb.squeeze(kb.tanh(kb.dot(x,self.W)+self.b),axis=-1)
        at=kb.softmax(et)
        at=kb.expand_dims(at,axis=-1)
        output=x*at
        return kb.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [16]:
lstmAtt_model = Sequential()
lstmAtt_model.add(embedding_layer)
lstmAtt_model.add(LSTM(16, return_sequences=True))
lstmAtt_model.add(attention())
lstmAtt_model.add(Dense(2, activation="softmax"))

lstmAtt_model.compile(loss="categorical_crossentropy", optimizer="adagrad", metrics=["accuracy"])
lstmAtt_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 50)           5009900   
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 16)           4288      
_________________________________________________________________
attention_2 (attention)      (None, 16)                216       
_________________________________________________________________
dense (Dense)                (None, 2)                 34        
Total params: 5,014,438
Trainable params: 5,014,438
Non-trainable params: 0
_________________________________________________________________


In [17]:
x_train_index = tokenizer.texts_to_sequences(x_train)
x_train_index = pad_sequences(x_train_index, maxlen=200)

In [18]:
y_train_categorical = keras.utils.to_categorical(y_train, 2)

In [19]:
lstmAtt_model.fit(x_train_index[:35000], y_train_categorical[:35000], epochs=10, batch_size=32, verbose=1, validation_data=(x_train_index[35000:], y_train_categorical[35000:]))

Epoch 1/10
1094/1094 [==============================] - 79s 60ms/step - loss: 0.6950 - accuracy: 0.5200 - val_loss: 0.6855 - val_accuracy: 0.5767
Epoch 2/10
1094/1094 [==============================] - 65s 59ms/step - loss: 0.6843 - accuracy: 0.5909 - val_loss: 0.6808 - val_accuracy: 0.6219
Epoch 3/10
1094/1094 [==============================] - 65s 59ms/step - loss: 0.6802 - accuracy: 0.6257 - val_loss: 0.6760 - val_accuracy: 0.6547
Epoch 4/10
1094/1094 [==============================] - 65s 60ms/step - loss: 0.6750 - accuracy: 0.6606 - val_loss: 0.6697 - val_accuracy: 0.6853
Epoch 5/10
1094/1094 [==============================] - 65s 59ms/step - loss: 0.6679 - accuracy: 0.6933 - val_loss: 0.6607 - val_accuracy: 0.7079
Epoch 6/10
1094/1094 [==============================] - 63s 58ms/step - loss: 0.6586 - accuracy: 0.7008 - val_loss: 0.6469 - val_accuracy: 0.7122
Epoch 7/10
1094/1094 [==============================] - 69s 63ms/step - loss: 0.6427 - accuracy: 0.7105 - val_loss: 0.6279 -

In [20]:
x_test_index = tokenizer.texts_to_sequences(x_test)
x_test_index = pad_sequences(x_test_index, maxlen=200)

In [21]:
y_pred = lstmAtt_model.predict(x_test_index)

In [22]:
print(classification_report(y_test, np.argmax(y_pred, axis=1).astype("float32")))

              precision    recall  f1-score   support

           0       0.73      0.69      0.71      2558
           1       0.69      0.73      0.71      2442

    accuracy                           0.71      5000
   macro avg       0.71      0.71      0.71      5000
weighted avg       0.71      0.71      0.71      5000



In [23]:
lstmAtt_model.save("IMDB_LSTMAtt")

INFO:tensorflow:Assets written to: IMDB_LSTMAtt/assets
INFO:tensorflow:Assets written to: IMDB_LSTMAtt/assets
